In [302]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [458]:
round(np.mean([1,1,1,1,1,0,0,0,0,0]))

0

In [464]:
np.unique([1,1,1,1,1])

array([1])

In [492]:
class Tree:

    def __init__(self, rand=None,
                 get_candidate_columns=all_columns,
                 min_samples=2):
        self.rand = rand  # for replicability
        self.get_candidate_columns = get_candidate_columns  # needed for random forests
        self.min_samples = min_samples

    def build(self, X: np.array, y: np.array):
        """
        Recusrively build a tree, stop recursion when a split has a child node with gini impurity 0 or
        when we have less than min_samples samples.
        """
        # should probably add some checks that the target vector entires are 1 and 0.
        
        # just for robustness and testing
        assert (len(X) == len(y)), "The input data and label vector are not of equal length" 
        
        if (len(y) < self.min_samples): # we are in a leaf node
            return TreeNode(None, None, round(np.mean(y))) # make the majority class the prediction for this node
        if (np.all(y == 1)): # check if we have a node with all ones
            return TreeNode(None, None, 1)
        if (np.all(y == 0)):
            return TreeNode(None, None, 0)
        
        right_i, left_i, decision_rule = find_decision_rule(X, y)
        
        left_subtree = Tree()
        right_subtree = Tree()
        
        
        return TreeNode(left_subtree.build(X[left_i], y[left_i]),right_subtree.build(X[right_i], y[right_i]), decision_rule) 


    def find_decision_rule(X, y):
        """
        Input: X - data, y - labels
        Output: A tuple (left, right, decision_rule), left indicies, right indicies and rule. Rule itself is a tuple
        of the index of the feature to split on and the value of where to split.)
        """

        decision_rule = None
        best_info_gain = 0 
        
        for i, feature in enumerate(X.T): # we iterate over rows of the transpose so it's actually over columns
            # TODO: better implementation, this just uses the mean to split
            split_value = np.mean(feature) # this is just a number of where to split
            left = np.where(feature < split_value)[0]
            right = np.where(feature >= split_value)[0]
            
            
            current_info_gain = information_gain(y, left, right)
            if (current_info_gain > best_info_gain):
                best_info_gain = current_info_gain
                decision_rule = (i, split_value)
        
        return (left, right, decision_rule) 
    
    def gini_impurity(y):
        """
        We can use this simplified version because we are solving a strictly binary classification problem, 
        assume y is a numpy array with values of 0 or 1.
        """
        
        label_one_probability = sum(y)/len(y)
        
        return 1 - ((label_one_probability)**2 + (1-label_one_probability)**2)
    
    def information_gain(y , left_partition_indicies, right_partition_indicies):
        n_left = len(left_partition_indicies)
        n_right = len(right_partition_indicies)
        n = n_left + n_right
        
        l_weight = n_left/n
        r_weight = n_right/n
        
        inf_gain = (gini_impurity(y[np.concatenate((left_partition_indicies,right_partition_indicies))]) 
                    - gini_impurity(y[left_partition_indicies])*l_weight 
                    - gini_impurity(y[right_partition_indicies])*r_weight)
        
        return inf_gain

In [593]:
class TreeNode:
    
    def __init__(self, left, right, decision_rule):
        """Left and right are TreeNode objects. Decision rule is either a tuple with a feature and value 
        to split on or a single value which determines the leaf's predicted label.
        """
        self.left = left
        self.right = right
        self.decision_rule = decision_rule

    def predict(self, X):
        prediction = np.empty(len(X))
        
        if ((self.left is None) and (self.right is None)): # we are in a leaf node
            return self.decision_rule
        
        # get left and right indices
        left_i = np.where(X.T[self.decision_rule[0]] < self.decision_rule[1])
        right_i = np.where(X.T[self.decision_rule[0]] >= self.decision_rule[1])
        
        left_prediction = self.left.predict(X[left_i])
        right_prediction = self.right.predict(X[right_i])
        
        prediction[left_i] = left_prediction
        prediction[right_i] = right_prediction
               
        return prediction

In [510]:
#  def predict(self, X):
#         split_feature = self.split_col
#         threshold = self.threshold
#         node_prediction = np.array(range(len(X)), dtype='<U16')

#         #left = X[X[:, split_feature] < threshold, :]   #to gre v left node
#         #right = X[X[:, split_feature] >= threshold, :]   # to gre v right node

#         if self.Rnode != None and self.Lnode != None:
#             left = X[X[:, split_feature] < threshold, :]  # to gre v left node
#             right = X[X[:, split_feature] >= threshold, :]  # to gre v right node
#             left_leaf = self.Lnode.predict(left)
#             right_leaf = self.Rnode.predict(right)
#             node_prediction[X[:, split_feature] < threshold] = left_leaf
#             node_prediction[X[:, split_feature] >= threshold] = right_leaf
#             return node_prediction

#         else:
#             node_prediction[:] = self.majority
#             return node_prediction

### todo: implement the proper split algo

In [603]:
T = Tree()

In [604]:
root_node = T.build(X,y)

In [605]:
root_node.decision_rule

(75, 0.0007780754433206924)

In [607]:
root_node.predict(X)

array([0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
       0.])

In [609]:
np.equal(y, root_node.predict(X))

array([ True, False,  True, False, False, False,  True, False,  True,
       False, False,  True,  True, False, False,  True, False, False,
        True,  True,  True, False, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True,  True, False,
        True, False,  True, False, False,  True, False,  True, False,
       False, False, False,  True, False,  True, False,  True,  True,
       False,  True, False, False,  True,  True, False,  True, False,
       False,  True,  True,  True,  True, False, False,  True, False,
       False, False,  True, False,  True, False, False, False,  True,
        True,  True,  True, False, False,  True, False, False,  True,
       False,  True,  True, False, False, False,  True,  True,  True,
       False,  True, False,  True, False,  True,  True, False, False,
        True,  True,  True, False,  True, False, False,  True, False,
       False, False,  True, False, False, False,  True,  True,  True,
        True,  True,

In [298]:
X, y = np.array(data)[:,0:-1], np.array(data)[:,-1]

In [355]:
X1 = np.array([[1, 2],[2, 3],[3, 4],[4, 8]])
y1 = np.array([0,0,1,1])

In [371]:
np.unique(y1)

array([0, 1])

In [356]:
X1[find_decision_rule(np.array([[1, 2],[2, 3],[3, 4],[4, 8]]), np.array([0,0,0,1]))[0]]

array([[1, 2],
       [2, 3],
       [3, 4]])

In [346]:
find_decision_rule(np.array(data)[:,0:-1], np.array(data)[:,-1])

(array([  1,   3,   4,   5,   9,  10,  11,  12,  15,  16,  18,  20,  24,
         25,  27,  28,  30,  36,  40,  41,  43,  44,  46,  48,  50,  51,
         53,  55,  56,  58,  63,  65,  69,  70,  72,  74,  77,  78,  79,
         80,  81,  82,  83,  84,  85,  87,  88,  95,  97,  98,  99, 100,
        103, 104, 106, 107, 108, 110, 113, 115, 116, 119, 120, 122, 123,
        127, 128, 129, 131, 132, 134, 135, 136, 138, 139, 140, 141, 142,
        143, 144, 150, 151, 152, 154, 155, 156, 157, 158, 162, 163, 164,
        171, 173, 177, 178, 179, 180, 182], dtype=int64),
 array([  0,   2,   6,   7,   8,  13,  14,  17,  19,  21,  22,  23,  26,
         29,  31,  32,  33,  34,  35,  37,  38,  39,  42,  45,  47,  49,
         52,  54,  57,  59,  60,  61,  62,  64,  66,  67,  68,  71,  73,
         75,  76,  86,  89,  90,  91,  92,  93,  94,  96, 101, 102, 105,
        109, 111, 112, 114, 117, 118, 121, 124, 125, 126, 130, 133, 137,
        145, 146, 147, 148, 149, 153, 159, 160, 161, 165, 166, 167

In [175]:
data = pd.read_csv("tki-resistance.csv")

In [177]:
data["Class"] = data["Class"].map({"Bcr-abl":0, "Wild type":1})

In [178]:
gini_impurity(np.array(data.Class))

0.49637845178813933

In [179]:
information_gain(np.array(data.Class), range(0,100), range(100,188))

0.0011560146508085845

In [111]:
gini_impurity(np.array([[1,1],[1,1],[1,0],[1,0]]), -1)

0.5

In [118]:
gini_impurity(np.array([[1,1],[1,0]]), -1)*0.5

0.25

In [119]:
gini_impurity(np.array([[1,1],[1,0]]), -1)*0.5

0.25

In [157]:
information_gain(np.array([0,0,1,1]), [0,1], [2,3])

0.5

In [158]:
information_gain(np.array([0,0,1,1]), [0,2], [1,3])

0.0

In [174]:
data

array([[-1.98536943e-03,  2.67660747e-03,  1.20055629e-03, ...,
         1.50547850e-04,  2.24156092e-04,  0.00000000e+00],
       [-1.66334053e-03,  1.72126861e-03,  1.16556826e-03, ...,
         1.72792087e-04,  7.23331654e-05,  1.00000000e+00],
       [-1.61161679e-03,  3.21999952e-03,  1.81925538e-03, ...,
         8.51674570e-05,  4.15107928e-04,  0.00000000e+00],
       ...,
       [ 2.52499461e-03,  2.90831930e-03,  1.41831616e-03, ...,
         3.15575800e-04,  3.31320754e-04,  0.00000000e+00],
       [ 2.60736546e-03,  2.07337927e-03,  1.82516431e-03, ...,
         1.08573548e-04,  2.16675215e-04,  1.00000000e+00],
       [ 3.16547062e-03,  3.55755669e-03,  5.95305421e-04, ...,
         1.79665012e-04,  3.57525341e-04,  0.00000000e+00]])

In [79]:
np.array(data)[range(1,100)]

array([[ 1.77009070e-03,  1.72126861e-03,  1.16556826e-03, ...,
         1.72792087e-04,  7.23331654e-05,  1.00000000e+00],
       [-5.06886652e-04,  3.21999952e-03,  1.81925538e-03, ...,
         8.51674570e-05,  4.15107928e-04,  0.00000000e+00],
       [ 7.37330934e-04,  4.07839090e-03,  1.47835684e-03, ...,
         2.94718737e-04,  9.90693731e-05,  0.00000000e+00],
       ...,
       [ 8.62824689e-04, -6.88554865e-04,  1.21599798e-03, ...,
         4.81905602e-05, -1.80551734e-04,  0.00000000e+00],
       [ 1.36631281e-04,  5.34343806e-04,  1.70834486e-03, ...,
         4.18673918e-05, -2.99019339e-05,  0.00000000e+00],
       [ 1.67960011e-04,  7.15732558e-04,  2.07812878e-03, ...,
         7.30009040e-05,  1.17486230e-04,  1.00000000e+00]])

In [ ]:
for feature in data:
    

In [269]:
find_decision_rule(d[:,0:-1], d[:,-1])

0.1884708916009439


(75, 0.0007780754433206924)

In [240]:
np.concatenate(a,b, axi)

TypeError: only integer scalar arrays can be converted to a scalar index

In [228]:
d.shape

(188, 199)

In [229]:
d[:,-1].shape

(188,)

In [230]:
d[:,0:-1].shape

(188, 198)

In [219]:
rule = np.mean()

TypeError: _mean_dispatcher() missing 1 required positional argument: 'a'

In [217]:
for i, feature in enumerate(d.T):
    print(i)
    print(feature)

0
[-4.99224257e-04  1.77009070e-03 -5.06886652e-04  7.37330934e-04
 -1.38101784e-04  9.61597400e-04 -8.25685972e-04 -8.30130653e-05
 -1.28442763e-03  1.88658712e-03  8.08963431e-04 -7.95562303e-04
  7.28199972e-04 -1.35749068e-03 -1.26041836e-03 -6.40352400e-04
  9.46867313e-04 -7.13843480e-04 -1.23379917e-03 -4.19140194e-04
  1.37521357e-03  1.39092615e-03  2.66182344e-04  3.16547062e-03
  1.33347184e-03  9.97209147e-04 -8.16471559e-04 -5.93344565e-04
  1.72006619e-04 -1.09410686e-03  8.51133948e-04  8.26618449e-04
 -3.81219449e-04 -2.17027514e-04  1.02789128e-03 -8.17742743e-05
  9.52495545e-04  1.98527892e-03  5.37582327e-04 -1.19434546e-03
 -2.99634244e-04  7.09550861e-04 -1.00864042e-03 -1.12503527e-04
  2.52499461e-03  3.73303840e-04  4.13343294e-04  1.65157442e-03
  1.93650409e-03  1.24119935e-03 -5.05145671e-04  1.04400018e-03
  9.51297026e-04 -1.19327592e-03  6.95137874e-04 -1.66334053e-03
  2.34016054e-03 -6.60185884e-04  8.75193524e-04 -2.24811462e-04
  1.62183822e-05 -1.434

 0.01282151 0.01328862]
178
[0.01012655 0.01027869 0.01078894 0.01067592 0.01083545 0.01082709
 0.01012575 0.0102556  0.00986836 0.01114594 0.00814475 0.01020471
 0.01037036 0.0107256  0.01005238 0.00994823 0.01053505 0.01000216
 0.01062332 0.00937913 0.01072222 0.01030266 0.01035533 0.01009369
 0.01053467 0.00837652 0.01050219 0.01038521 0.00909008 0.00997532
 0.00762232 0.01013112 0.01123077 0.01057349 0.00954526 0.00984162
 0.00966371 0.00934037 0.01120422 0.00991295 0.01049039 0.01052737
 0.01010142 0.00946242 0.01009593 0.01088094 0.00983487 0.01037166
 0.01127681 0.00995298 0.01038377 0.01075246 0.01047799 0.00622215
 0.00993048 0.01130017 0.01108752 0.01092299 0.00956969 0.00915886
 0.01063082 0.01032357 0.00989743 0.00855189 0.0102624  0.01023241
 0.00984377 0.01019067 0.01020345 0.00894899 0.01009991 0.01011696
 0.0106941  0.01028573 0.01023819 0.010216   0.01002485 0.01078393
 0.00991745 0.0107978  0.01074211 0.00896554 0.01035596 0.00870757
 0.00945173 0.00957635 0.00950822 

In [208]:
np.mean(d.T[0])

0.0003355731439158787

In [214]:
np.mean(d.T[0])
np.where(d.T[0] < np.mean(d.T[0]))

(array([  0,   2,   4,   6,   7,   8,  11,  13,  14,  15,  17,  18,  19,
         22,  26,  27,  28,  29,  32,  33,  35,  39,  40,  42,  43,  50,
         53,  55,  57,  59,  60,  61,  64,  65,  66,  68,  71,  72,  73,
         75,  76,  77,  79,  80,  85,  86,  89,  96,  98,  99, 102, 107,
        111, 117, 118, 121, 122, 125, 127, 129, 130, 134, 137, 143, 147,
        148, 149, 153, 154, 156, 157, 158, 159, 160, 161, 164, 168, 175,
        176, 178, 179, 182, 184, 186], dtype=int64),)